In [1]:
import numpy as np
import matplotlib.cm as cm
from matplotlib import ticker
import math
import scipy
from scipy import spatial
import matplotlib.pyplot as plt
import matplotlib
import xarray as xr
import dask
from sklearn.neighbors import KDTree
import netCDF4
from metpy import calc
from metpy.units import units
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy import stats

from sklearn.cluster import AgglomerativeClustering
from mpl_toolkits.axes_grid1 import make_axes_locatable
import scipy.cluster.hierarchy as shc
from scipy.cluster.hierarchy import dendrogram, linkage

In [2]:
fz = 15*1.5
lw = 4
siz = 100
XNNA = 1.25 # Abscissa where architecture-constrained network will be placed
XTEXT = 0.25 # Text placement
YTEXT = 0.3 # Text placement

plt.rc('text', usetex=False)
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
#mpl.rcParams["font.serif"] = "STIX"
plt.rc('font', family='serif', size=fz)
matplotlib.rcParams['lines.linewidth'] = lw

### Change the paths to the scalar and test data

In [ ]:
Test_Images = np.load("/fast/gmooers/Preprocessed_Data/W_Variable/Trackable_Space_Time_W_Test.npy")

Max_Scalar = np.load("/fast/gmooers/Preprocessed_Data/Centered_50_50/Space_Time_Max_Scalar.npy")
Min_Scalar = np.load("/fast/gmooers/Preprocessed_Data/Centered_50_50/Space_Time_Min_Scalar.npy")

Test_Images = np.interp(Test_Images, (0, 1), (Min_Scalar, Max_Scalar))

### Need paths to two different latent spaces:

- 1024 D (No PCA applied)
- 2D (Post-PCA)

In [4]:
z_vector = np.load("Reduced_Data_For_Agg/z_1024_31.npy")
z_test_tsne_track = np.load("Reduced_Data_For_Agg/z_31.npy")

### Need to make a Figures Directory

- 31 refers to the config file so I would change that to match your confog file (easier to keep track of the VAES)

In [ ]:
def fancy_dendrogram(*args, **kwargs):
    fig, ax = plt.subplots(figsize=(30,10))
    max_d = kwargs.pop('max_d', None)
    if max_d and 'color_threshold' not in kwargs:
        kwargs['color_threshold'] = max_d
    annotate_above = kwargs.pop('annotate_above', 0)

    ddata = dendrogram(*args, **kwargs)

    if not kwargs.get('no_plot', False):
        plt.title('Hierarchical Clustering Dendrogram (truncated)')
        plt.xlabel('sample index or (cluster size)')
        plt.ylabel('distance')
        for i, d, c in zip(ddata['icoord'], ddata['dcoord'], ddata['color_list']):
            x = 0.5 * sum(i[1:3])
            y = d[1]
            if y > annotate_above:
                plt.plot(x, y, 'o', c=c)
                plt.annotate("%.3g" % y, (x, y), xytext=(0, -5),
                             textcoords='offset points',
                             va='top', ha='center')
        if max_d:
            plt.axhline(y=max_d, c='k')
    plt.savefig("Figures/Full_Day_Half_31.png")
    return ddata

In [ ]:
X = z_vector
Z = linkage(X, 'ward')

In [ ]:
fancy_dendrogram(
Z,
    truncate_mode='lastp',
    p=10,
    leaf_rotation=90.,
    leaf_font_size=15.,
    show_contracted=False,
    annotate_above=100,  # useful in small plots so annotations don't overlap
)
plt.show()
#plt.savefig("Figures/Z_Dedrogram.png")

### Need to make a compressed data directory

In [ ]:
cluster = AgglomerativeClustering(n_clusters=2, affinity='euclidean', linkage='ward')  
cluster.fit_predict(z_vector)
labels_2 = cluster.labels_
np.save("Compressed_Data/Full_Day_Half_31_2.npy",labels_2)

In [ ]:
cmap = matplotlib.colors.ListedColormap(["green", "blue"])
fig, ax = plt.subplots(figsize=(30,10))

cp = ax.scatter(x=z_test_tsne_track[:, 0], y=z_test_tsne_track[:, 1], c=cluster.labels_, cmap=cmap, s=10.0)
ax.set_title("Agglomerative Clustering", fontsize = fz*0.75)
cbar = fig.colorbar(cp)
cbar.set_label(label="Clusters", rotation="horizontal", fontsize=fz*0.5, labelpad=30, y = 0.55)
cbar.set_ticks([0,1])
cbar.set_ticklabels(["Group 1","Group 2"])
plt.savefig("Figures/Full_Day_Half_31_2.png")

In [ ]:
cluster = AgglomerativeClustering(n_clusters=3, affinity='euclidean', linkage='ward')  
cluster.fit_predict(z_vector)
labels_3 = cluster.labels_
np.save("Compressed_Data/Full_Day_Half_31_3.npy",labels_3)

In [ ]:
cmap = matplotlib.colors.ListedColormap(["yellow","green", "blue"])
fig, ax = plt.subplots(figsize=(30,10))

cp = ax.scatter(x=z_test_tsne_track[:, 0], y=z_test_tsne_track[:, 1], c=cluster.labels_, cmap=cmap, s=10.0)
ax.set_title("Agglomerative Clustering", fontsize = fz*0.75)
cbar = fig.colorbar(cp)
cbar.set_label(label="Clusters", rotation="horizontal", fontsize=fz*0.5, labelpad=30, y = 0.55)
cbar.set_ticks([0,1,2])
cbar.set_ticklabels(["Group 1","Group 2","Group 3"])
plt.savefig("Figures/Full_Day_Half_31_3.png")

In [ ]:
cluster = AgglomerativeClustering(n_clusters=4, affinity='euclidean', linkage='ward')  
cluster.fit_predict(z_vector)
labels_4 = cluster.labels_
np.save("Compressed_Data/Full_Day_Half_31_4.npy",labels_4)

In [ ]:
cluster = AgglomerativeClustering(n_clusters=5, affinity='euclidean', linkage='ward')  
cluster.fit_predict(z_vector)
labels_5 = cluster.labels_
np.save("Compressed_Data/Full_Day_Half_31_5.npy",labels_5)

In [ ]:
cmap = matplotlib.colors.ListedColormap(["red","orange","yellow","green", "blue"])
fig, ax = plt.subplots(figsize=(30,10))

cp = ax.scatter(x=z_test_tsne_track[:, 0], y=z_test_tsne_track[:, 1], c=cluster.labels_, cmap=cmap, s=10.0)
ax.set_title("Agglomerative Clustering", fontsize = fz*0.75)
cbar = fig.colorbar(cp)
cbar.set_label(label="Clusters", rotation="horizontal", fontsize=fz*0.5, labelpad=30, y = 0.55)
cbar.set_ticks([0,1,2,3,4])
cbar.set_ticklabels(["Group 1","Group 2","Group 3","Group 4","Group 5"])
plt.savefig("Figures/Full_Day_Half_31_5.png")

In [ ]:
cluster = AgglomerativeClustering(n_clusters=6, affinity='euclidean', linkage='ward')  
cluster.fit_predict(z_vector)
labels_6 = cluster.labels_
np.save("Compressed_Data/Full_Day_Half_31_6.npy",labels_6)

In [ ]:
cmap = matplotlib.colors.ListedColormap(["red","orange","yellow","green", "blue","purple"])
fig, ax = plt.subplots(figsize=(30,10))

cp = ax.scatter(x=z_test_tsne_track[:, 0], y=z_test_tsne_track[:, 1], c=cluster.labels_, cmap=cmap, s=10.0)
ax.set_title("Agglomerative Clustering", fontsize = fz*0.75)
cbar = fig.colorbar(cp)
cbar.set_label(label="Clusters", rotation="horizontal", fontsize=fz*0.5, labelpad=30, y = 0.55)
cbar.set_ticks([0,1,2,3,4,5])
cbar.set_ticklabels(["Group 1","Group 2","Group 3","Group 4","Group 5","Group 6"])
plt.savefig("Figures/Full_Day_Half_31_6.png")

In [ ]:
cluster = AgglomerativeClustering(n_clusters=8, affinity='euclidean', linkage='ward')  
cluster.fit_predict(z_vector)
labels_8 = cluster.labels_
np.save("Compressed_Data/Full_Day_Half_31_8.npy",labels_8)

In [ ]:
cmap = matplotlib.colors.ListedColormap(["pink","red","orange","yellow","green", "blue","purple","black"])
fig, ax = plt.subplots(figsize=(30,10))

cp = ax.scatter(x=z_test_tsne_track[:, 0], y=z_test_tsne_track[:, 1], c=cluster.labels_, cmap=cmap, s=10.0)
ax.set_title("Agglomerative Clustering", fontsize = fz*0.75)
cbar = fig.colorbar(cp)
cbar.set_label(label="Clusters", rotation="horizontal", fontsize=fz*0.5, labelpad=30, y = 0.55)
cbar.set_ticks([0,1,2,3,4,5,6,7])
cbar.set_ticklabels(["Group 1","Group 2","Group 3","Group 4","Group 5","Group 6","Group 7","Group 8"])
plt.savefig("Figures/Full_Day_Half_31_8.png")

# 3 Groups

In [ ]:
path = "/DFS-L/DATA/pritchard/gmooers/Raw_Data_Storage/MAPS/SPCAM/100_Days/New_SPCAM5/archive/TimestepOutput_Neuralnet_SPCAM_216/atm/hist/TimestepOutput_Neuralnet_SPCAM_216.cam.h1.2009-01-20-00000.nc"
extra_variables = xr.open_dataset(path)
ha = extra_variables.hyai.values
hb = extra_variables.hybi.values
PS = 1e5
Pressures_real = PS*ha+PS*hb

In [ ]:
group = {}
mean = {}
std = {}
for igroup in range(3):
    group[igroup] = Test_Images[labels_3==igroup,:,:]
    mean[igroup] = np.mean(group[igroup],axis=0)
    std[igroup] = np.std(group[igroup],axis=0)

In [ ]:
fz = 15
lw = 4
siz = 100
XNNA = 1.25 # Abscissa where architecture-constrained network will be placed
XTEXT = 0.25 # Text placement
YTEXT = 0.3 # Text placement

plt.rc('text', usetex=False)
matplotlib.rcParams['mathtext.fontset'] = 'stix'
matplotlib.rcParams['font.family'] = 'STIXGeneral'
#mpl.rcParams["font.serif"] = "STIX"
plt.rc('font', family='serif', size=fz)
matplotlib.rcParams['lines.linewidth'] = lw

In [ ]:
vmax_mean = 1.0
vmin_mean = -1.0
vmax_std = 1.0
vmin_std = 0.0
cmap="RdBu_r"
var_labels = ["Group 1", "Group 2", "Group 3"]
fig, ax = plt.subplots(nrows=3,ncols=3,figsize=(22,16))
for igroup in range(3):
    axes = ax[igroup]
    cs = axes[0].pcolor(Xs, Zs, mean[igroup], cmap=cmap, vmax = 0.01, vmin = -0.01)
    divider = make_axes_locatable(axes[0])
    cax = divider.append_axes("right", size="5%", pad=0.05)
    fig.colorbar(cs, cax=cax)
    cs = axes[1].pcolor(Xs, Zs, std[igroup], cmap=cmap, vmax = vmax_std, vmin = vmin_std)
    divider = make_axes_locatable(axes[1])
    cax = divider.append_axes("right", size="5%", pad=0.05)
    fig.colorbar(cs, cax=cax)
    cs = axes[2].plot(np.flipud(np.nanmean(std[igroup],axis=1)), Pressures_real[:-1]/100.0)
    axes[2].set_xlim(0,0.5)
    axes[0].set_ylim(axes[0].get_ylim()[::-1])
    axes[1].set_ylim(axes[1].get_ylim()[::-1])
    axes[2].set_ylim(axes[2].get_ylim()[::-1])
    if igroup == 2:
        axes[0].set_xlabel('CRMs')
        axes[1].set_xlabel('CRMs')
        axes[2].set_xlabel('m/s')
    if igroup == 1:
        axes[0].set_ylabel('Pressure')
        axes[1].set_ylabel('m/s')
        axes[1].yaxis.set_label_coords(1.21,0.5)
        #axes[1].yaxis.set_label_position("right")
        #axes[1].yaxis.tick_right()
    axes[1].set_yticks([])
    axes[2].set_yticks([])
    if igroup < 2:
        axes[0].set_xticks([])
        axes[1].set_xticks([])
        axes[2].set_xticks([])
    if igroup == 0:
        axes[0].set_title("Mean")
        axes[1].set_title("Standard Devation")
        axes[2].set_title("Zonal Mean Standard Devation")
        
plt.suptitle("Agglomerative Clustering Groups in 2D Latent Space Projection", y = 0.93)
plt.savefig("Figures/Full_Day_Half_31_Z_3_Group_Comparison_Agglomerative_Labels.png")

# 5 Groups

In [ ]:
group = {}
mean = {}
std = {}
for igroup in range(5):
    group[igroup] = Test_Images[labels_5==igroup,:,:]
    mean[igroup] = np.mean(group[igroup],axis=0)
    std[igroup] = np.std(group[igroup],axis=0)

In [ ]:
vmax_mean = 1.0
vmin_mean = -1.0
vmax_std = 1.0
vmin_std = 0.0
cmap="RdBu_r"
var_labels = ["Group 1", "Group 2", "Group 3"]
fig, ax = plt.subplots(nrows=5,ncols=3,figsize=(22,16))
for igroup in range(5):
    axes = ax[igroup]
    cs = axes[0].pcolor(Xs, Zs, mean[igroup], cmap=cmap, vmax = 0.01, vmin = -0.01)
    divider = make_axes_locatable(axes[0])
    cax = divider.append_axes("right", size="5%", pad=0.05)
    fig.colorbar(cs, cax=cax)
    cs = axes[1].pcolor(Xs, Zs, std[igroup], cmap=cmap, vmax = vmax_std, vmin = vmin_std)
    divider = make_axes_locatable(axes[1])
    cax = divider.append_axes("right", size="5%", pad=0.05)
    fig.colorbar(cs, cax=cax)
    cs = axes[2].plot(np.flipud(np.nanmean(std[igroup],axis=1)), Pressures_real[:-1]/100.0)
    axes[2].set_xlim(0,0.75)
    axes[0].set_ylim(axes[0].get_ylim()[::-1])
    axes[1].set_ylim(axes[1].get_ylim()[::-1])
    axes[2].set_ylim(axes[2].get_ylim()[::-1])
    if igroup == 4:
        axes[0].set_xlabel('CRMs')
        axes[1].set_xlabel('CRMs')
        axes[2].set_xlabel('m/s')
    if igroup == 2:
        axes[0].set_ylabel('Pressure')
        axes[1].set_ylabel('m/s')
        axes[1].yaxis.set_label_coords(1.21,0.5)
        #axes[1].yaxis.set_label_position("right")
        #axes[1].yaxis.tick_right()
    axes[1].set_yticks([])
    axes[2].set_yticks([])
    if igroup < 4:
        axes[0].set_xticks([])
        axes[1].set_xticks([])
        axes[2].set_xticks([])
    if igroup == 0:
        axes[0].set_title("Mean")
        axes[1].set_title("Standard Devation")
        axes[2].set_title("Zonal Mean Standard Devation")
        
plt.suptitle("Agglomerative Clustering Groups in 2D Latent Space Projection", y = 0.93)
plt.savefig("Figures/Full_Day_Half_31_Z_5_Group_Comparison_Agglomerative_Labels.png")

# 6 Groups

In [ ]:
group = {}
mean = {}
std = {}
for igroup in range(6):
    group[igroup] = Test_Images[labels_6==igroup,:,:]
    mean[igroup] = np.mean(group[igroup],axis=0)
    std[igroup] = np.std(group[igroup],axis=0)

In [ ]:
vmax_mean = 1.0
vmin_mean = -1.0
vmax_std = 1.0
vmin_std = 0.0
cmap="RdBu_r"
var_labels = ["Group 1", "Group 2", "Group 3"]
fig, ax = plt.subplots(nrows=6,ncols=3,figsize=(22,16))
for igroup in range(6):
    axes = ax[igroup]
    cs = axes[0].pcolor(Xs, Zs, mean[igroup], cmap=cmap, vmax = 0.01, vmin = -0.01)
    divider = make_axes_locatable(axes[0])
    cax = divider.append_axes("right", size="5%", pad=0.05)
    fig.colorbar(cs, cax=cax)
    cs = axes[1].pcolor(Xs, Zs, std[igroup], cmap=cmap, vmax = vmax_std, vmin = vmin_std)
    divider = make_axes_locatable(axes[1])
    cax = divider.append_axes("right", size="5%", pad=0.05)
    fig.colorbar(cs, cax=cax)
    cs = axes[2].plot(np.flipud(np.nanmean(std[igroup],axis=1)), Pressures_real[:-1]/100.0)
    axes[2].set_xlim(0,0.75)
    axes[0].set_ylim(axes[0].get_ylim()[::-1])
    axes[1].set_ylim(axes[1].get_ylim()[::-1])
    axes[2].set_ylim(axes[2].get_ylim()[::-1])
    if igroup == 5:
        axes[0].set_xlabel('CRMs')
        axes[1].set_xlabel('CRMs')
        axes[2].set_xlabel('m/s')
    if igroup == 3:
        axes[0].set_ylabel('Pressure')
        axes[1].set_ylabel('m/s')
        axes[1].yaxis.set_label_coords(1.21,0.5)
        #axes[1].yaxis.set_label_position("right")
        #axes[1].yaxis.tick_right()
    axes[1].set_yticks([])
    axes[2].set_yticks([])
    if igroup < 5:
        axes[0].set_xticks([])
        axes[1].set_xticks([])
        axes[2].set_xticks([])
    if igroup == 0:
        axes[0].set_title("Mean")
        axes[1].set_title("Standard Devation")
        axes[2].set_title("Zonal Mean Standard Devation")
        
plt.suptitle("Agglomerative Clustering Groups in 2D Latent Space Projection", y = 0.93)
plt.savefig("Figures/Full_Day_Half_31_Z_6_Group_Comparison_Agglomerative_Labels.png")